## Imports and settings

In [1]:
import pandas as pd
import sys
sys.path.append('../src/data/')
import util as util

In [8]:
setup, dset = util.read_setup_and_data()
validated = util.validate_setup_and_data(setup, dset)

if not validated:
    print("ERROR: setup and data files are not valid.")
    raise KeyboardInterrupt

# replace dset columns with setup keys
dset.columns = setup.keys()

# remove respondents who did not agree to share data
nrsp_raw = dset.shape[0]
dset = dset[dset.q1 == "Yes"]

print(f"Questions  : {dset.shape[1]}")
print(f"Respondents - all           : {nrsp_raw}")
print(f"Respondents - agreed sharing: {dset.shape[0]}")

Questions  : 20
Respondents - all           : 36
Respondents - agreed sharing: 36


,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20
Timestamp,,,,,,,,,,,,,,,,,,,,
12/12/2022 10:32:07,Yes,Diego Alonso Álvarez,RSE Team (or Imperial Central RSE Team),The main RSE Group for the university/research...,Team Lead,2017.0,IT/Research IT,1.0,8.0,6.0,2.0,2.0,"1: industry, 1: an embedded RSE team within Im...",5.0,Demand significantly greater than the number o...,"Hiring processes are long-winded, so we're alw...",75%,"No. We aim for 70%, but it is not a hard bound...",15%,NaN
12/12/2022 10:42:05,Yes,Mark Turner,Newcastle Research Software Engineering,The main RSE Group for the university/research...,Head of RSE,2018.0,Independent Institute/Facility,2.0,20.0,5.0,0.0,3.0,"One to industry, two to other institutions",6.0,Demand significantly greater than the number o...,"We have funding, but we can't find RSEs to hir...",80%,80%,<5%,NaN
12/12/2022 11:04:31,Yes,Adam Tyson,SWC/GCNU Neuroinformatics Unit (UCL),An RSE Group for a specific subset/group withi...,Head Research Engineer,2022.0,Academic Department (supporting a specific dom...,1.0,4.0,4.0,0.0,0.0,NaN,3.0,Demand greater than the number of RSEs,We lack the funding to make new positions avai...,0,None,100%,NaN
12/12/2022 11:05:56,Yes,James Graham,King's e-Research RSE (placeholder name),The main RSE Group for the university/research...,Head of RSE,2022.0,IT/Research IT,1.0,1.0,1.0,0.0,0.0,NaN,4.0,Demand greater than the number of RSEs,"Hiring processes are long-winded, so we're alw...",100,70 (TBC),20 (TBC),Might be useful to distinguish between IT and ...
12/12/2022 12:55:10,Yes,Martin Robinson,Oxford Research Software Engineering (OxRSE),The main RSE Group for the university/research...,Head of RSE,2019.0,Independent Institute/Facility,3.0,5.0,4.0,0.0,3.0,1) Senior Lecturer position 2) position in ind...,0.0,Demand significantly greater than the number o...,We lack the funding to make new positions avai...,100%,100%,0%,We are currently applying for central universi...
12/12/2022 13:23:06,Yes,Martin O'Reilly,The Research Engineering Group at The Alan Tur...,The main RSE Group for the university/research...,Director,2016.0,Central team separate from both IT and the Ins...,3.0,44.0,17.0,2.0,10.0,1 internal move to domain-focussed data scienc...,17.0,Demand significantly greater than the number o...,NaN,All posts are permanent with the exception of ...,70-75% against funded projects.,15-20%,"We're 34 people in post now, but have 10 peopl..."
12/12/2022 14:40:00,Yes,Christopher Edsall,Cambridge RSE,The main RSE Group for the university/research...,Head of Research Software Engineering,2013.0,IT/Research IT,6.0,10.0,2.0,0.0,3.0,"research in Europe (2x), industry (1x)",14.0,Demand significantly greater than the number o...,"We can attract RSEs, but can't hire them becau...",50%,90%,10%,This response covers three subgroups (core tea...
12/12/2022 14:41:26,Yes,Alex Coleman,Research Computing at the University of Leeds,The main RSE Group for the university/research...,Research Software Engineer,2019.0,IT/Research IT,4.0,5.0,0.0,0.0,2.0,One became a teaching fellow in the School of ...,2.0,Demand greater than the number of RSEs,"Hiring processes are long-winded, so we're alw...",100%,0,80%,We currently do not have a head of RSE at Leed...
12/12/2022 15:30:58,Yes,Jonathan Cooper,UCL Advanced Research Computing,The main RSE Group for the university/research...,Head of Research Software Engineering,2012.0,We are a hybrid professional services / academ...,3.0,40.0,20.0,2.0,6.0,"1 postdoc, 3 industry (1 SE, 1 training, 1 RSE...",18.0,Demand significantly greater than the number o...,"Hiring processes are long-winded, so we're alw...",100%,"Not explicitly, but ~15% is core funded","~15% service to others, ~15% internal (Mgmt, A...","We are not just an RSE group any more, but pro..."
